<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>

<h1 align=center><font size = 5>Assignment: SQL Notebook for Peer Assignment</font></h1>

Estimated time needed: **60** minutes.

## Introduction
Using this Python notebook you will:

1.  Understand the Spacex DataSet
2.  Load the dataset  into the corresponding table in a IBM Db2 database
4.  Execute SQL queries to answer assignment questions 


## Overview of the DataSet

SpaceX has gained worldwide attention for a series of historic milestones (marcos históricos). 

It is the only private company ever to return a spacecraft from low-earth orbit, which it first accomplished in December 2010.
SpaceX advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars wheras other providers cost upward of 165 million dollars each, much of the savings is because Space X can reuse the first stage. 

Therefore if we can determine if the first stage will land, we can determine the cost of a launch. 

This dataset includes a record for every payload carried on a SpaceX mission into outer space.


### Download the datasets

This assignment requires you to load the SpaceX dataset.

In many cases the dataset to be analyzed is available as a .CSV (comma separated values) file, perhaps on the internet. Click on the link below to download and save the dataset (.CSV file):

 <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv" target="_blank">Spacex DataSet</a>



In [10]:
!pip install sqlalchemy==1.3.9  # Comando de terminal, não é Python

  Using cached SQLAlchemy-1.3.9-cp312-cp312-linux_x86_64.whl
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.43
    Uninstalling SQLAlchemy-2.0.43:
      Successfully uninstalled SQLAlchemy-2.0.43
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.3.9 which is incompatible.
jupyterhub 5.2.1 requires SQLAlchemy>=1.4.1, but you have sqlalchemy 1.3.9 which is incompatible.


🧠 SQLAlchemy — em poucas palavras

É uma biblioteca Python para trabalhar com bases de dados relacionais (suporte parcial de bases de dados NoSQL via extensões).

💬 O que faz exatamente

Permite-te:

- Ligar o Python a uma base de dados (ex.: SQLite, MySQL, PostgreSQL, etc.);

- Ler e escrever dados nessas bases de dados (fazer SELECT, INSERT, UPDATE, DELETE);

- Gerir tabelas e relações sem teres de escrever diretamente código SQL — se quiseres, podes usar objetos Python.

⚙️ Duas formas de usar

SQLAlchemy tem duas “camadas” principais:

1️⃣ Core (camada SQL)

Usas comandos parecidos com SQL, mas escritos em Python.
👉 É como escrever SQL, mas com sintaxe Pythonica.

2️⃣ ORM (Object Relational Mapper)

Transforma tabelas em classes Python e linhas em objetos.
👉 Assim, trabalhas com a base de dados como se fosse um conjunto de objetos.

🎯 Em resumo
| SQLAlchemy serve para...         | Em termos simples                          |
| -------------------------------- | ------------------------------------------ |
| Ligar o Python a bases de dados  | Faz a ponte entre Python e bases de dados  |
| Escrever queries SQL em Python   | Podes usar `text()` ou o ORM               |
| Criar e gerir tabelas            | Sem escrever SQL manualmente               |
| Manter o código limpo e portável | Funciona com vários tipos de base de dados |


💡 Pensa nela como o “tradutor” entre Python e SQL.
Em vez de falares diretamente “em SQL”, o SQLAlchemy traduz o que o teu código Python quer fazer para comandos que a base de dados entende.

### Connect to the database

Let us first load the SQL extension and establish a connection with the database


In [11]:
!pip install ipython-sql prettytable

  Using cached sqlalchemy-2.0.43-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.6 kB)
Using cached sqlalchemy-2.0.43-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.3 MB)
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.3.9
    Uninstalling SQLAlchemy-1.3.9:
      Successfully uninstalled SQLAlchemy-1.3.9


In [12]:
# Carrega a extensão ipython-sql no notebook.
%load_ext sql 

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [13]:
import csv, sqlite3  
import prettytable
prettytable.DEFAULT = 'DEFAULT'

# sqlite3 é o módulo Python para SQLite, incluído na biblioteca padrão. Só funciona com bases de dados SQLite, porque foi feito especificamente para esse motor.
con = sqlite3.connect("my_data1.db") # Cria uma conexão com a base de dados my_data1.db. A extensão .db é apenas uma convenção. Se o ficheiro não existir, o SQLite cria automaticamente a base de dados.
# O resultado (con) é um objeto da classe Connection que permite executar comandos SQL e gerir transações.

cur = con.cursor() # cur é um novo objeto da classe Cursor — que é o que usas para executar queries SQL (cur.execute(...)).

Nota:
Apenas SQLite funciona com um único ficheiro cujo nome e extensão podes escolher.
As restantes bases de dados funcionam como servidores — a extensão dos ficheiros internos não interessa, só interages com elas via cliente/driver ou SQL.

| Base de dados     | Ficheiro físico                       | Observações                                            |
| ----------------- | ------------------------------------- | ------------------------------------------------------ |
| SQLite            | 1 ficheiro (.db, .sqlite)             | Extensão é só convenção                                |
| MySQL/PostgreSQL  | Vários ficheiros internos do servidor | Não há extensão “.db” acessível diretamente            |
| Oracle/SQL Server | Vários ficheiros de dados e logs      | Extensão varia internamente, não usada pelo utilizador |

In [14]:
!pip install -q pandas # -q é uma opção quiet (silenciosa): Faz com que o pip não mostre mensagens durante a instalação, apenas erros importantes ou avisos críticos. Útil para evitar poluição do output.

%sql
- É um magic command do Jupyter fornecido pelo módulo ipython-sql.
- Permite executar instruções SQL diretamente numa célula do notebook, sem precisar criar Connection e Cursor manualmente.

In [15]:
%sql sqlite:///my_data1.db

# É uma string de ligação (connection string) que indica ao ipython-sql:
    # 1. sqlite → estamos a usar SQLite como motor de base de dados.
    # 2. ///my_data1.db → caminho para o ficheiro da base de dados local.
        # Três barras (///) indicam caminho relativo ao diretório atual do notebook.

In [16]:
import pandas as pd

In [17]:
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")

In [20]:
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False, method="multi") 
# Método do pandas que permite escrever o conteúdo de um DataFrame diretamente numa tabela SQL. Ou seja, transforma os dados do DataFrame em linhas de uma tabela da base de dados.
# "SPACEXTBL": Nome da tabela na base de dados onde os dados vão ser guardados. Se a tabela não existir, o pandas cria automaticamente.
# con: Conexão à base de dados (sqlite3.Connection)
# if_exists='replace': Controla o que acontece se a tabela já existir: 'replace' → a tabela antiga é substituída.
# index=False: Indica que o índice do DataFrame não será escrito como coluna na tabela SQL.
# method="multi": Controla como os inserts são feitos no SQL: "multi" → insere várias linhas de cada vez (mais rápido para DataFrames grandes). Alternativa: None → insere linha a linha.

# method=None → gera uma query INSERT para cada linha do DataFrame.
# INSERT INTO SPACEXTBL (col1, col2) VALUES (val1, val2);
# INSERT INTO SPACEXTBL (col1, col2) VALUES (val3, val4);
# Simples, mas mais lento se houver muitas linhas.

# method="multi" → agrupa várias linhas num único INSERT:
# INSERT INTO SPACEXTBL (col1, col2) VALUES (val1, val2), (val3, val4), (val5, val6);
# Mais eficiente, porque reduz o número de queries enviadas ao motor SQL.

101

In [22]:
# O Valor retornado é um inteiro que indica o número de linhas do DataFrame que foram inseridas na tabela SQL.

In [21]:
# DataFrame.to_sql(name, con, *, schema=None, if_exists='fail', index=True, index_label=None, chunksize=None, dtype=None, method=None)

# Parameters:
# name: str 
# Name of SQL table. 
#
# con: consqlalchemy.engine.(Engine or Connection) or sqlite3.Connection
# Using SQLAlchemy makes it possible to use any DB supported by that library. 
# Legacy support is provided for sqlite3.Connection objects. 
# The user is responsible for engine disposal and connection closure for the SQLAlchemy connectable. See here. 
# If passing a sqlalchemy.engine.Connection which is already in a transaction, the transaction will not be committed. 
# If passing a sqlite3.Connection, it will not be possible to roll back the record insertion.
#
# if_exists: {‘fail’, ‘replace’, ‘append’}, default ‘fail’
# How to behave if the table already exists.
# fail: Raise a ValueError.
# replace: Drop the table before inserting new values.
# append: Insert new values to the existing table.
#
# index: bool, default True
# Write DataFrame index as a column. Uses index_label as the column name in the table. Creates a table index for this column.
#
# index_label: str or sequence, default None
# Column label for index column(s). If None is given (default) and index is True, then the index names are used. A sequence should be given if the DataFrame uses MultiIndex.
#
# method: {None, ‘multi’, callable}, optional
# Controls the SQL insertion clause used:
# None : Uses standard SQL INSERT clause (one per row).
# ‘multi’: Pass multiple values in a single INSERT clause.
# callable with signature (pd_table, conn, keys, data_iter). Details and a sample callable implementation can be found in the section insert method.

**Note: This code below is added to remove blank rows from table**

In [36]:
# DROP THE TABLE IF EXISTS

%sql DROP TABLE IF EXISTS SPACEXTABLE;

 * sqlite:///my_data1.db
Done.


[]

In [37]:
%sql CREATE TABLE SPACEXTABLE as select * from SPACEXTBL where Date is not null  

# O asterisco * significa todas as colunas da tabela.

 * sqlite:///my_data1.db
Done.


[]

In [38]:
%sql SELECT * FROM SPACEXTABLE LIMIT 5

 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


## Tasks

Now write and execute SQL queries to solve the assignment tasks.

**Note: If the column names are in mixed case enclose it in double quotes
   For Example "Landing_Outcome"**

### Task 1




##### Display the names of the unique launch sites  in the space mission


In [49]:
%%sql 
SELECT DISTINCT 
    "Launch_Site" 
FROM SPACEXTABLE;

 * sqlite:///my_data1.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40



### Task 2


#####  Display 5 records where launch sites begin with the string 'CCA' 


In [47]:
%%sql 
SELECT * 
FROM SPACEXTABLE 
WHERE "Launch_Site" LIKE 'CCA%' 
LIMIT 5;

 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


### Task 3




##### Display the total payload mass carried by boosters launched by NASA (CRS)


In [51]:
%%sql 
SELECT 
    SUM("PAYLOAD_MASS__KG_") AS payload_sum
FROM SPACEXTABLE;

 * sqlite:///my_data1.db
Done.


payload_sum
619967


### Task 4




##### Display average payload mass carried by booster version F9 v1.1


In [53]:
%%sql 
SELECT AVG(PAYLOAD_MASS__KG_) AS avg_payload_mass 
FROM SPACEXTABLE
WHERE Booster_Version LIKE 'F9 v1.1%';	

 * sqlite:///my_data1.db
Done.


avg_payload_mass
2534.6666666666665


### Task 5

##### List the date when the first succesful landing outcome in ground pad was acheived.


_Hint:Use min function_ 


In [58]:
%%sql
SELECT DISTINCT "Landing_Outcome" FROM SPACEXTABLE;

 * sqlite:///my_data1.db
Done.


Landing_Outcome
Failure (parachute)
No attempt
Uncontrolled (ocean)
Controlled (ocean)
Failure (drone ship)
Precluded (drone ship)
Success (ground pad)
Success (drone ship)
Success
Failure


In [61]:
%%sql
SELECT MIN("DATE") AS "earliest sucess (ground pad)"
FROM SPACEXTABLE
WHERE "Landing_Outcome" = 'Success (ground pad)';    

 * sqlite:///my_data1.db
Done.


earliest sucess (ground pad)
2015-12-22


### Task 6

##### List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000


In [63]:
%%sql
SELECT Booster_Version
FROM SPACEXTABLE
WHERE 
    "Landing_Outcome" = 'Success (drone ship)' 
    AND "PAYLOAD_MASS__KG_" BETWEEN 4000 AND 6000;    

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 FT B1022
F9 FT B1026
F9 FT B1021.2
F9 FT B1031.2


### Task 7




##### List the total number of successful and failure mission outcomes


In [71]:
%%sql
SELECT 
    "Mission_Outcome",
    COUNT(*) AS total_missions
FROM SPACEXTABLE
GROUP BY "Mission_Outcome";

 * sqlite:///my_data1.db
Done.


Mission_Outcome,total_missions
Failure (in flight),1
Success,98
Success,1
Success (payload status unclear),1


### Task 8



##### List all the booster_versions that have carried the maximum payload mass, using a subquery with a suitable aggregate function.


In [72]:
%%sql
SELECT Booster_Version
FROM SPACEXTABLE
WHERE PAYLOAD_MASS__KG_ = (
    SELECT MAX(PAYLOAD_MASS__KG_)
    FROM SPACEXTABLE
);

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 B5 B1048.4
F9 B5 B1049.4
F9 B5 B1051.3
F9 B5 B1056.4
F9 B5 B1048.5
F9 B5 B1051.4
F9 B5 B1049.5
F9 B5 B1060.2
F9 B5 B1058.3
F9 B5 B1051.6


### Task 9


##### List the records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015.

**Note: SQLLite does not support monthnames. So you need to use  substr(Date, 6,2) as month to get the months and substr(Date,0,5)='2015' for year.**


In [88]:
%%sql

SELECT 
    SUBSTR(Date, 6,2) AS MONTH,
    "Booster_Version",
    "Launch_Site", 
    "Landing_Outcome"
FROM SPACEXTABLE
WHERE SUBSTR(Date,1,4) = '2015' AND "Landing_Outcome" = 'Failure (drone ship)';

 * sqlite:///my_data1.db
Done.


MONTH,Booster_Version,Launch_Site,Landing_Outcome
01,F9 v1.1 B1012,CCAFS LC-40,Failure (drone ship)
04,F9 v1.1 B1015,CCAFS LC-40,Failure (drone ship)


In [85]:
# SUBSTR(string, start, length), 
# string → texto ou coluna. 
# start → posição inicial (1=primeiro carácter).
# length (opcional) → número de caracteres a extrair.

### Task 10




##### Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order.


In [98]:
%%sql
SELECT 
    "Landing_Outcome", COUNT(*) AS n_landings 
FROM SPACEXTABLE
WHERE Date BETWEEN '2010-06-04' AND '2017-03-20'
GROUP BY "Landing_Outcome"
ORDER BY n_landings DESC;

 * sqlite:///my_data1.db
Done.


Landing_Outcome,n_landings
No attempt,10
Success (drone ship),5
Failure (drone ship),5
Success (ground pad),3
Controlled (ocean),3
Uncontrolled (ocean),2
Failure (parachute),2
Precluded (drone ship),1


In [97]:
# Em SQLite, mesmo que a coluna de datas seja TEXT (como neste caso), se estiver no formato YYYY-MM-DD, a comparação funciona lexicograficamente (alfabética), que coincide com a ordem cronológica.

In [95]:
%%sql
PRAGMA table_info(SPACEXTABLE);

 * sqlite:///my_data1.db
Done.


cid,name,type,notnull,dflt_value,pk
0,Date,TEXT,0,None,0
1,Time (UTC),TEXT,0,None,0
2,Booster_Version,TEXT,0,None,0
3,Launch_Site,TEXT,0,None,0
4,Payload,TEXT,0,None,0
5,PAYLOAD_MASS__KG_,INT,0,None,0
6,Orbit,TEXT,0,None,0
7,Customer,TEXT,0,None,0
8,Mission_Outcome,TEXT,0,None,0
9,Landing_Outcome,TEXT,0,None,0


### Reference Links

* <a href ="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20String%20Patterns%20-%20Sorting%20-%20Grouping/instructional-labs.md.html?origin=www.coursera.org">Hands-on Lab : String Patterns, Sorting and Grouping</a>  

*  <a  href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20Built-in%20functions%20/Hands-on_Lab__Built-in_Functions.md.html?origin=www.coursera.org">Hands-on Lab: Built-in functions</a>

*  <a  href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20Sub-queries%20and%20Nested%20SELECTs%20/instructional-labs.md.html?origin=www.coursera.org">Hands-on Lab : Sub-queries and Nested SELECT Statements</a>

*   <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Module%205/DB0201EN-Week3-1-3-SQLmagic.ipynb">Hands-on Tutorial: Accessing Databases with SQL magic</a>

*  <a href= "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Module%205/DB0201EN-Week3-1-4-Analyzing.ipynb">Hands-on Lab: Analyzing a real World Data Set</a>




## Author(s)

<h4> Lakshmi Holla </h4>


## Other Contributors

<h4> Rav Ahuja </h4>


<!--
## Change log
| Date | Version | Changed by | Change Description |
|------|--------|--------|---------|
| 2024-07-10 | 1.1 |Anita Verma | Changed Version|
| 2021-07-09 | 0.2 |Lakshmi Holla | Changes made in magic sql|
| 2021-05-20 | 0.1 |Lakshmi Holla | Created Initial Version |
-->


## <h3 align="center"> © IBM Corporation 2021. All rights reserved. <h3/>
